# DSO 560 Natural Language Processing Final Project

### Team members: Tanner Curley, Shao Xuan Chew, Robert Zhu


### Todo: overall code structure

In [1]:
# !python -m spacy download en_core_web_md

In [39]:
# perform lemmatization (not recommended when using any pretrained embedding)
LEMMA = False
# size of the vocab for
VOCAB_SIZE = 20000
# for self attention
DATA_POINTS = 10
DIMENSIONS = 240
SEQUENCE_LENGTH = 4
# use gpu
USE_GPU = 0

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17358218745217873517
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7806648320
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1175065762901300741
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:07:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [4]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
import keras
import tensorflow as tf
from keras.layers import Input, Dense, LSTM, Flatten, concatenate, Activation, RepeatVector, Permute
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
#from keras_self_attention import SeqSelfAttention
from random import randint
from numpy import array, argmax, asarray, zeros
import numpy as np 
import pandas as pd
import spacy
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from textacy import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
nlp = spacy.load('en_core_web_md')

In [6]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
df = pd.read_csv('complaints_processed.csv')
df.head()


,Unnamed: 0,product,narrative
0,0,credit_card,purchase order day shipping amount receive pro...
1,1,credit_card,forwarded message date tue subject please inve...
2,2,retail_banking,forwarded message cc sent friday pdt subject f...
3,3,credit_reporting,payment history missing credit report speciali...
4,4,credit_reporting,payment history missing credit report made mis...


In [8]:
complaint_types = list(df['product'].value_counts().index)

In [9]:
NUM_CLASSES = len(complaint_types)

0. Summary statistics

In [10]:
df.shape

(162421, 3)

In [11]:
df['product'].value_counts()

credit_reporting       91179
debt_collection        23150
mortgages_and_loans    18990
credit_card            15566
retail_banking         13536
Name: product, dtype: int64

Note that the dataset is imbalanced

1. Preprocessing
    1. remove hyphens and accents
    2. create customized stopwords list and remove stopwords
    3. regex cleaning
    4. typo correction with textblob
    5. lemmatization

In [12]:
df['narrative'] = df['narrative'].apply(str)

In [13]:
preproc = preprocessing.make_pipeline(
    preprocessing.normalize.hyphenated_words,
    preprocessing.remove.accents,
)

In [14]:
set(stopwords.words('english'))
# removing some negative words from stopwords list
nltk_stopwords = set(stopwords.words('english'))
nltk_stopwords.remove('below')
nltk_stopwords.remove("aren't")
nltk_stopwords.remove('couldn')
nltk_stopwords.remove("couldn't")
nltk_stopwords.remove("didn't")
nltk_stopwords = list(nltk_stopwords)
# add some abstract terms
nltk_stopwords.append('like')
nltk_stopwords.append('please')

nltk_stopwords = set(nltk_stopwords)

In [15]:
# removing all words with more than 15 digits
df['narrative'] = df['narrative'].str.replace(r'\b([a-z]|[A-Z]){15,}\b', '', case=False)

C:\Users\Robert\AppData\Local\Temp\ipykernel_4144\868481445.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['narrative'] = df['narrative'].str.replace(r'\b([a-z]|[A-Z]){15,}\b', '', case=False)


In [16]:
# removing all characters that appear more than 2 times
df['narrative'] = df['narrative'].str.replace(r'(.)\1{3,}?', '', case=False)


C:\Users\Robert\AppData\Local\Temp\ipykernel_4144\1200782744.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['narrative'] = df['narrative'].str.replace(r'(.)\1{3,}?', '', case=False)


In [17]:
# %pip install -U textblob
# !python -m textblob.download_corpora

In [18]:
from textblob import Word

In [19]:
from textblob import TextBlob

In [20]:
def spelling_correction(sentence: str):
    sentence = str(sentence)
    to_ret = TextBlob(sentence)
    to_ret = str(to_ret.correct())
    return to_ret

In [21]:
# df['narrative'] = df['narrative'].apply(spelling_correction)

In [22]:
lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(str(sentence)))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [23]:
if LEMMA:
    df['narrative'] = df['narrative'].apply(lemmatize_sentence)

In [24]:
labels = df['product']

2. model preparation
    1. tokenize
    2. integer encoding
    3. glove vectors
    4. embedding matrix
    

In [25]:
# tokenize text
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(df['narrative'])

In [26]:
# integer encoding
from typing import List
def get_max_token_length_per_doc(docs: List[List[str]])-> int:
    return max(list(map(lambda x: len(x.split()), docs)))

# get the max length in terms of token length
max_length = get_max_token_length_per_doc(df['narrative'])


def integer_encode_documents(docs, tokenizer):
    return tokenizer.texts_to_sequences(docs)

from keras.preprocessing.sequence import pad_sequences
MAX_SEQUENCE_LENGTH = 300
# integer encode the documents
encoded_docs = integer_encode_documents(df['narrative'], tokenizer)
# this is a list of lists, the numbers represent the index position of that word.
# for instance, 33 means the 33rd word in the vocabulary
padded_docs = pad_sequences(encoded_docs, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [27]:
# glove vectors
def load_glove_vectors():
    embeddings_index = {}
    with open('glove\glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [28]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((VOCAB_SIZE, 100))
for word, i in tokenizer.word_index.items():
    if i >= VOCAB_SIZE:
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

3. model definition
    1. LSTM
    2. Bi-directional LSTM
    3. RNN
    4. BERT
    5. Self-Attention

In [29]:
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers import Flatten, Masking
from keras.utils.vis_utils import plot_model

def make_lstm_classification_model(plot=False):
    model = keras.models.Sequential()
    model.add(Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(units=64, input_shape=(1, MAX_SEQUENCE_LENGTH)))
    model.add(Dense(16))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    
    # Compile the model
    model.compile(loss="categorical_crossentropy", optimizer= "adam", metrics=['accuracy'])
    # summarize the model
    model.summary()
    
    if plot:
        plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [30]:
model = make_lstm_classification_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          2000000   
_________________________________________________________________
masking (Masking)            (None, 300, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
dense (Dense)                (None, 16)                1040      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 85        
Total params: 2,043,365
Trainable params: 43,365
Non-trainable params: 2,000,000
_________________________________________________________________


In [31]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
transfomed_labels = encoder.fit_transform(labels)
# print(transfomed_labels)

In [47]:
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, transfomed_labels, test_size=0.2)

In [48]:
# fit the model
if not USE_GPU:
    with tf.device('/cpu:0'):
        history = model.fit(padded_docs, transfomed_labels, validation_split = 0.1, epochs=10, verbose=1)
else:
        history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=10, verbose=1)

Epoch 1/10
4569/4569 [==============================] - 1144s 250ms/step - loss: 0.4048 - accuracy: 0.8565 - val_loss: 0.4352 - val_accuracy: 0.8420
Epoch 2/10
4569/4569 [==============================] - 1174s 257ms/step - loss: 0.3645 - accuracy: 0.8703 - val_loss: 0.4136 - val_accuracy: 0.8484
Epoch 3/10
4569/4569 [==============================] - 1215s 266ms/step - loss: 0.3386 - accuracy: 0.8794 - val_loss: 0.4127 - val_accuracy: 0.8514
Epoch 4/10
4569/4569 [==============================] - 1249s 273ms/step - loss: 0.3188 - accuracy: 0.8857 - val_loss: 0.4087 - val_accuracy: 0.8527
Epoch 5/10
4569/4569 [==============================] - 1286s 281ms/step - loss: 0.3007 - accuracy: 0.8932 - val_loss: 0.4018 - val_accuracy: 0.8561
Epoch 6/10
4569/4569 [==============================] - 26073s 6s/step - loss: 0.2855 - accuracy: 0.8980 - val_loss: 0.4208 - val_accuracy: 0.8532
Epoch 7/10
4569/4569 [==============================] - 1386s 303ms/step - loss: 0.2729 - accuracy: 0.9029 -

In [ ]:
import matplotlib.pyplot as plt # From https://keras.io/visualization/


def plot_performance(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

plot_performance(history)